In [1]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [14]:
print("Data read started...")
data = pd.read_csv("result_tetris.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(603, 11)


# Eliminate EEG data

In [15]:
for i in range (1,5):
    data = np.delete(data, 1, 1) 

data.shape

(603, 7)

In [16]:
data[1]

array(['tetris1', 349860.0, 1688.7, 676.15, 4.378305084745762, 335.19,
       537.78], dtype=object)

# Dictionary for the levels

In [17]:
level=["tetris1","tetris2","tetris3"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [18]:
for i in range (1,7):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [19]:
for i in range(len(data)):
    data[i, 0] = level2int[data[i, 0]]

x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [20]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


# Shuffle data

In [21]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [22]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [23]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(422, 6) (422, 3) (90, 6) (90, 3) (91, 6) (91, 3)


# Build the net

In [24]:
model = Sequential()

model.add(Dense(200, input_shape=(6,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

In [25]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 422 samples, validate on 90 samples
Epoch 1/150
422/422 [==============================] - 0s - loss: 0.7594 - acc: 0.6682 - val_loss: 0.5401 - val_acc: 0.7333
Epoch 2/150
422/422 [==============================] - 0s - loss: 0.5013 - acc: 0.7370 - val_loss: 0.4463 - val_acc: 0.7556
Epoch 3/150
422/422 [==============================] - 0s - loss: 0.4412 - acc: 0.7654 - val_loss: 0.4240 - val_acc: 0.7444
Epoch 4/150
422/422 [==============================] - 0s - loss: 0.4613 - acc: 0.7370 - val_loss: 0.4138 - val_acc: 0.8000
Epoch 5/150
422/422 [==============================] - 0s - loss: 0.4254 - acc: 0.7559 - val_loss: 0.4014 - val_acc: 0.7778
Epoch 6/150
422/422 [==============================] - 0s - loss: 0.4125 - acc: 0.7701 - val_loss: 0.3993 - val_acc: 0.7889
Epoch 7/150
422/422 [==============================] - 0s - loss: 0.3953 - acc: 0.7962 - val_loss: 0.3921 - val_acc: 0.7667
Epoch 8/150
422/422 [==============================] - 0s - loss: 0.4056 - acc: 0.7678 

In [26]:
model.evaluate(x_test, y_test, batch_size=batch_size)

50/91 [===============>..............] - ETA: 0s

[0.2026680493420297, 0.9230769099770012]

In [28]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(tetris1)', 'class 1(tetris2)', 'class 2(tetris3)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

32/91 [=========>....................] - ETA: 0s[1 0 2 1 0 2 2 0 0 0 1 2 2 0 2 2 1 2 2 2 2 0 2 2 0 1 1 0 2 2 1 0 2 2 0 0 2
 1 0 1 2 0 0 2 1 2 0 2 2 1 2 1 2 2 1 1 2 2 1 2 0 2 2 1 2 0 1 2 0 1 2 1 0 0
 1 0 2 0 0 1 1 1 0 2 1 0 0 2 2 2 1]
32/91 [=========>....................] - ETA: 0s                  precision    recall  f1-score   support

class 0(tetris1)       1.00      1.00      1.00        27
class 1(tetris2)       0.92      0.82      0.87        28
class 2(tetris3)       0.87      0.94      0.91        36

     avg / total       0.92      0.92      0.92        91

[[27  0  0]
 [ 0 23  5]
 [ 0  2 34]]
